# Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install

In [ ]:
!pip install -U datasets bitsandbytes accelerate transformers peft trl

# 7.4 QLoRA 미세조정

## 모델

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
)
import torch

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1"
)

gen_cfg = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<bos><start_of_turn>user
북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?<end_of_turn>
<start_of_turn>model
북태평양 기단과 오호츠크 해양 기단은 이 지역의 온난성을 형성하는 주요 기단입니다. 북태평양 기단은 이 지역에서 가장 강한 기단으로서, 북쪽으로 이동하여 태평양에 도달할 때까지 오호츠크 해양 기단을 따라 이동합니다. 그런 다음 북태평양 기단은 이 지역에 머무르며 지속적으로 온난성을 형성합니다. 이때, 오호츠크 해양 기단은 북태평양 기단과 결합되어 온난성을 형성할 수 있습니다. 이 두 기단은 전체적으로 지속적으로 온난성을 형성하고, 이 지역의 온난성을 형성하는 기간은 겨울철 기단과 여름철 기단에 따라 달라집니다.<eos>


## 데이터셋

In [1]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")

doc = dataset["train"]['question'][0]
messages = [{"role": "user", "content": doc}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
)

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [ ]:
def preprocess(example):
    return tokenizer.apply_chat_template(
        [
            {"role": "user", "content": example['question']},
            {"role": "model", "content": example['answers']['text'][0]},
        ],
        return_dict=True,
    )

dataset = dataset.map(
    preprocess, 
    num_proc=2, 
    remove_columns=dataset["train"].column_names
)

## QLoRA 미세조정 준비

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=bnb_config,
)
model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    add_special_tokens=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## QLoRA 학습

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling 


args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Books/outputs/ckpt",
    max_steps=3000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    warmup_steps=0.03,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    data_collator=collator,
)

trainer.train()

## 모델 저장

In [10]:
model = model.merge_and_unload()
model.save_pretrained("/content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
!ls -alh /content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1

total 4.7G
drwx------ 2 root root 4.0K May 26 13:35 backup
-rw------- 1 root root  713 May 26 13:37 config.json
-rw------- 1 root root  132 May 26 13:37 generation_config.json
-rw------- 1 root root 4.7G May 26 13:38 model-00001-of-00002.safetensors
-rw------- 1 root root  65M May 26 13:38 model-00002-of-00002.safetensors
-rw------- 1 root root  14K May 26 13:38 model.safetensors.index.json


## 추론

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
import torch

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1")
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    add_special_tokens=True,
)

# 이전에 만들었던 inputs, gen_cfg 사용

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))